In [ ]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
r = requests.get('https://httpbin.org/user-agent')
useragent = r.json()['user-agent']
headers = {'User-Agent': useragent,
           'From': 'frv3fp@virginia.edu'}

In [4]:
url = 'https://www.rottentomatoes.com/browse/movies_in_theaters/sort:a_z?page=5'
r = requests.get(url, headers=headers)
r

<Response [200]>

In [6]:
mysoup = BeautifulSoup(r.text, 'html.parser')

In [7]:
mysoup

<!DOCTYPE html>

<html dir="ltr" lang="en" prefix="fb: http://www.facebook.com/2008/fbml og: http://opengraphprotocol.org/schema/" xmlns="http://www.w3.org/1999/xhtml">
<head prefix="og: http://ogp.me/ns# flixstertomatoes: http://ogp.me/ns/apps/flixstertomatoes#">
<script charset="UTF-8" crossorigin="anonymous" data-domain-script="7e979733-6841-4fce-9182-515fac69187f" integrity="sha384-TKdmlzVmoD70HzftTw4WtOzIBL5mNx8mXSRzEvwrWjpIJ7FZ/EuX758yMDWXtRUN" src="https://cdn.cookielaw.org/consent/7e979733-6841-4fce-9182-515fac69187f/otSDKStub.js" type="text/javascript">
</script>
<script type="text/javascript">
                function OptanonWrapper() { }
            </script>
<script ccpa-opt-out-geo="US" ccpa-opt-out-ids="USP" ccpa-opt-out-lspa="false" charset="UTF-8" src="https://cdn.cookielaw.org/opt-out/otCCPAiab.js" type="text/javascript">
</script>
<script src="/assets/pizza-pie/javascripts/bundles/roma/rt-common.js?single"></script>
<meta content="text/html; charset=utf-8" http-equiv=

<a data-track="scores" data-qa="discovery-media-list-item-caption" href="/m/a_complete_unknown" slot="caption" >

Anatomy of an HTML line:

*Tag: code that represents a specific visual thing to put on the page. "a" tage is for hyperlinks. 
<a> is our tag
Tags are often (not always) closed by typing something like </a>

*Attributes: parameters contained within a tag. Her it is: data-track="scores" data-qa="discovery-media-list-item-caption" href="/m/a_complete_unknown" slot="caption"

*Navigable string: whatever is typed between the opening and closing tags
<a> navigable string </a>

In [10]:
movielist = mysoup.find_all('a', attrs={'data-track': 'scores'})

In [13]:
len(movielist)
'https://www.rottentomatoes.com' + movielist[2]['href']

'https://www.rottentomatoes.com/m/a_knights_war'

In [14]:
#list comprehension: a loop over items in a list, outputs a different list
x = [1,2,3,4,5]

# [stuff I want to do for each item in the list for token in list]

[t**2 / 4 for t in x]

[0.25, 1.0, 2.25, 4.0, 6.25]

In [15]:
links = ['https://www.rottentomatoes.com' + m['href'] for m in movielist]
links

['https://www.rottentomatoes.com/m/2121',
 'https://www.rottentomatoes.com/m/a_complete_unknown',
 'https://www.rottentomatoes.com/m/a_knights_war',
 'https://www.rottentomatoes.com/m/a_real_pain',
 'https://www.rottentomatoes.com/m/all_we_imagine_as_light',
 'https://www.rottentomatoes.com/m/anora',
 'https://www.rottentomatoes.com/m/are_you_there_2024',
 'https://www.rottentomatoes.com/m/armand',
 'https://www.rottentomatoes.com/m/attack_on_titan_the_last_attack',
 'https://www.rottentomatoes.com/m/becoming_led_zeppelin',
 'https://www.rottentomatoes.com/m/blades_in_the_darkness',
 'https://www.rottentomatoes.com/m/bring_them_down',
 'https://www.rottentomatoes.com/m/captain_america_brave_new_world',
 'https://www.rottentomatoes.com/m/companion_2025',
 'https://www.rottentomatoes.com/m/conclave',
 'https://www.rottentomatoes.com/m/creation_of_the_gods_ii_demon_force',
 'https://www.rottentomatoes.com/m/dark_nuns',
 'https://www.rottentomatoes.com/m/den_of_thieves_2_pantera',
 'https:

In [18]:
# Delve into A Complete Unkown
# Title

url = 'https://www.rottentomatoes.com/m/a_complete_unknown'
r = requests.get(url, headers=headers)
mysoup = BeautifulSoup(r.text, 'html.parser')

In [25]:
titletag = mysoup.find('h1', attrs={'class': 'unset'})
title = titletag.text.strip()
mydict = {'title': title}
mydict

{'title': 'A Complete Unknown'}

In [29]:
descriptag = mysoup.find('rt-text', attrs={'slot': 'content'})
description = descriptag.text.strip()
mydict['description'] = description
mydict

{'title': 'A Complete Unknown',
 'description': "New York, 1961. Against the backdrop of a vibrant music scene and tumultuous cultural upheaval, an enigmatic 19-year-old from Minnesota arrives with his guitar and revolutionary talent, destined to change the course of American music. He forges intimate relationships with music icons of Greenwich Village on his meteoric rise, culminating in a groundbreaking and controversial performance that reverberates worldwide. Timothée Chalamet stars and sings as Bob Dylan in James Mangold's A COMPLETE UNKNOWN, the electric true story behind the rise of one of the most iconic singer-songwriters in history."}

In [30]:
def scrape_one(url):
    mydict = {}
    r = requests.get(url, headers=headers)
    mysoup = BeautifulSoup(r.text, 'html.parser')
 
    mydict['Title'] = mysoup.find('h1', 'unset').text.strip()
 
    sum = mysoup.find('meta', attrs={'name':'description'})
    mydict['Synopsis'] = sum['content']
 
    consensus = mysoup.find('div', 'consensus')
    try:
        mydict['Critic Consensus'] = consensus.p.text
    except:
        mydict['Critic Consensus'] = ''
 
    mydict['Critics Score'] = mysoup.find('rt-text', slot="criticsScore").text.strip()
    mydict['Audience Score'] = mysoup.find('rt-text', slot="audienceScore").text.strip()
 
    cat = mysoup.find_all('div',"category-wrap")
    cats = [x.text.strip().split('\n\n\n') for x in cat]
    for c in cats:
        mydict[c[0]] = c[1].replace('\n','')
 
    return mydict

In [31]:
scrape_one(links[0])

{'Title': '2121',
 'Synopsis': '100 years from now when humans are forced underground by an ecological disaster and live in bunkers throughout the world. Newborns are prized, middle-aged people are good for labor and the elderly are harvested. When a woman has an unplanned pregnancy, her mother is now in jeopardy and she must figure out how to help her mother escape before the harvesters come to collect her organs....',
 'Critic Consensus': '',
 'Critics Score': '',
 'Audience Score': '',
 'Director': 'Serpil Altin',
 'Producer': 'Korhan Ugur, Serpil Altin',
 'Screenwriter': 'Korhan Ugur, Serpil Altin',
 'Distributor': 'Indican Pictures',
 'Genre': 'Mystery & Thriller, Drama, Sci-Fi',
 'Original Language': 'English',
 'Release Date (Theaters)': 'Jan 31, 2025, Limited',
 'Box Office (Gross USA)': '$7.9K',
 'Runtime': '1h 32m'}

In [32]:
full_movie_list = [scrape_one(l) for l in links]

In [34]:
pd.DataFrame.from_records(full_movie_list)

,Title,Synopsis,Critic Consensus,Critics Score,Audience Score,Director,Producer,Screenwriter,Distributor,Genre,Original Language,Release Date (Theaters),Box Office (Gross USA),Runtime,Production Co,Rating,Sound Mix,Aspect Ratio,Release Date (Streaming),Rerelease Date (Theaters)
0,2121,100 years from now when humans are forced unde...,,,,Serpil Altin,"Korhan Ugur, Serpil Altin","Korhan Ugur, Serpil Altin",Indican Pictures,"Mystery & Thriller, Drama, Sci-Fi",English,"Jan 31, 2025, Limited",$7.9K,1h 32m,NaN,NaN,NaN,NaN,NaN,NaN
1,A Complete Unknown,"New York, 1961. Against the backdrop of a vibr...",Charged by Timothée Chalamet's electric perfor...,80%,96%,James Mangold,"Fred Berger, Bob Bookman, Timothée Chalamet, A...","James Mangold, Jay Cocks",Searchlight Pictures,"Biography, Drama, Music",English,"Dec 25, 2024, Wide",$69.0M,2h 21m,"Searchlight Pictures, The Picture Company, Ver...",R (Language),"Dolby Atmos, Dolby Digital",Digital 2.39:1,NaN,NaN
2,A Knight's War,A fearless knight braves a deadly realm to sav...,,,,Matthew Ninaber,Matthew Ninaber,Matthew Ninaber,DREAD,Fantasy,English,"Feb 7, 2025, Limited",NaN,1h 44m,High Rise Studio,NaN,NaN,NaN,"Feb 11, 2025",NaN
3,A Real Pain,Mismatched cousins David (Jesse Eisenberg) and...,Led by a scene-stealing turn from Kieran Culki...,96%,81%,Jesse Eisenberg,"Jesse Eisenberg, Ali Herting, Dave McCary, Ewa...",Jesse Eisenberg,Searchlight Pictures,"Comedy, Drama",English,"Nov 15, 2024, Wide",$8.3M,1h 29m,"Topic Studios, Extreme Emotions, Fruit Tree",R (Some Drug Use|Language Throughout),NaN,NaN,"Dec 31, 2024",NaN
4,All We Imagine as Light,"The light, the lives, and the textures of cont...",Capturing the here and now of modern India wit...,100%,67%,Payal Kapadia,NaN,Payal Kapadia,Sideshow / Janus Films,Drama,Malayalam,"Nov 15, 2024, Limited",$1.0M,1h 58m,"Les Films Fauves, BALDR Film, Petit Chaos, art...",NaN,Dolby Digital,Flat (1.66:1),"Feb 4, 2025",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,Vermiglio,"The lush and breathtaking beauty of the Alps, ...","Painterly and patient, Vermiglio carefully obs...",94%,,Maura Delpero,"Carole Baraton, Tatjana Kozar, Francesca Andreoli",Maura Delpero,Sideshow / Janus Films,Drama,Italian,"Dec 25, 2024, Limited",$158.2K,1h 59m,"RAI Cinema, Cinedora, Versus Production, Charades",NaN,NaN,NaN,NaN,NaN
64,When I'm Ready,As asteroids threaten to wipe out all life on ...,,,,Andrew Johnson,"Andrew Ortenberg, Eli Samek, Jordan Dykstra, R...",Andrew Ortenberg,Quiver Distribution,"Drama, Romance",English,"Feb 7, 2025, Limited",NaN,1h 48m,"19th Hole Productions, Film Bridge International",NaN,NaN,NaN,NaN,NaN
65,Wicked,"Wicked, the untold story of the witches of Oz,...",Defying gravity with its magical pairing of Cy...,88%,95%,Jon M. Chu,"Marc Platt, David Stone","Winnie Holzman, Dana Fox",Universal Pictures,"Kids & Family, Musical, Fantasy, Adventure",English,"Nov 22, 2024, Wide",$471.9M,2h 40m,Marc Platt Productions,PG (Some Scary Action|Brief Suggestive Materia...,Dolby Atmos,Digital 2.39:1,"Dec 31, 2024",NaN
66,Wolf Man,From Blumhouse and visionary writer-director L...,Director Leigh Whannell's attempt at bringing ...,51%,56%,Leigh Whannell,"Jason Blum, Ryan Gosling","Leigh Whannell, Corbett Tuck, Lauren Schuker B...",Universal Pictures,"Horror, Mystery & Thriller",English,"Jan 17, 2025, Wide",$20.6M,1h 43m,"Blumhouse Productions, Universal Pictures, Way...",R (Grisly Images|Bloody Violent Content|Some L...,Dolby Atmos,Digital 2.39:1,"Feb 4, 2025",NaN
